In [53]:
# Read csv to Dictionary
import csv
reader = csv.reader(open('unit_conversions.csv', 'r'), delimiter=';')
dict_unit_csv = {} # start with empty dictionary

####
## Add code to check whether two columns or not
## Add code for input types (string, float)
####

for k, v in reader:
    dict_unit_csv[k] = v # add key and value to dict

print(dict_unit_csv) # test output

{'distance_SI_km': '0,001', 'distance_SI_cm': '100', 'distance_SI_mm': '1000'}


In [54]:
# Convert Units
########
# Self-made Dictionary

# All conversions are made to SI units
# https://en.wikipedia.org/wiki/United_States_customary_units#Units_of_length

dict_unit_hardcoded = {
    # Distance to Meter (SI) using * multiplication to go to SI
    'distance_SI_km':0.001, # kilometer
    'distance_SI_m':1, # meter
    'distance_SI_cm':100, # centimeter
    'distance_SI_mm':1000, # millimeter
    'distance_USCS_mi.':1609.344, # miles
    'distance_USCS_ft':0.3048, # feet
    'distance_USCS_in':0.0254, # inch
    # Volume to Liter (SI) using * multiplication to go to SI
    'volume_USCS_cu_in':0.016387064, # cubic inch
    'volume_USCS_cu_ft':28.316846592, # cubic feet
    'volume_USCS_cu_yd':764.554857984, # cubic yard
    'volume_USCS_bbl':158.987294928, # oil barrel
    'volume_SI_L':1, # Liter
    # Temperatures
    'temperatures_USCS_°F': lambda x : ((5/9) * (x - 32)), # Fahrenheit to C
    'temperatures_USCS_F': lambda x : ((5/9) * (x - 32)), # Fahrenheit to C
    'temperatures_SI_°K': lambda x : (x - 273.15), # Kelvin
    'temperatures_SI_K': lambda x : (x - 273.15), # Kelvin
    'temperatures_SI_°C': 1, # Celsius
    'temperatures_SI_C': 1, # Celsius
    # Weights
    'mass_USCS_lb': 0.45359237, # Pounds
    'mass_SI_kg': 1, # Kilogram
    'mass_SI_g': 1000, # grams
}

In [55]:
# Make universal dictionary reader for unit conversions

def convert_units_from_dict(dict_to_use, unit_subject, unit_system, unit_specs, data_in):
    '''
    Retrieves conversion units from dictionary to be multiplied\n
    Apply a function in case it is a function\n
    dict_to_use = dictionary used to retrieve value\n
    unit_subject = describes the subject of unit, e.g. temperature, length\n
    unit_system = describes the unit systeml USCS, Imperial, SI etc\n
    unit_specs = Specificies the exact unit used
    '''
    retrieval_value = unit_subject + '_' + unit_system + '_' + unit_specs
    if retrieval_value not in dict_to_use:
        # This will stop the program!
        raise RuntimeError("Unit "  + retrieval_value + " not found in dictionary. Please update data or dictionary.")

    x = dict_to_use[retrieval_value]
    if callable(x):
        return x(data_in) # applies the function as stated in dictionary
    else:
        return data_in * x # conversion is not a number, e.g. a function


In [56]:
def cleanup_data_values_return_float(data_in):

    if type(data_in) is bool:
        raise RuntimeError("Datatype value in database is a boolean but it must be a floating point or integer")

    if type(data_in) is str:
        data_in = data_in.replace(',','.')
        if data_in.count('.') > 1:
            data_in = data_in.replace('.','', data_in.count('.') - 1)
    
    try:
        data_in = float(data_in)
    except:
        raise RuntimeError("Datatype value in database is a string but it must be a floating point or integer")

    return(float(data_in))

In [57]:
# General importing
import pandas as pd

# import
df_import_supplier_A = pd.DataFrame()
df_import_supplier_A = pd.read_csv("Data.csv", header=None, sep=';', encoding='utf-8', low_memory=False)

# Clean up the database
# drop column
try:
    df_import_supplier_A = df_import_supplier_A.drop(columns=range(1,7), axis=1)
    df_import_supplier_A = df_import_supplier_A.drop(columns=range(8,9), axis=1)
    df_import_supplier_A = df_import_supplier_A.drop(columns=range(10,14), axis=1)
except:
    pass

# Make the conversion list
df_products = pd.DataFrame() # This can become a loading option later
'''
df_products["12NC"] = df_import_supplier_A[0]
df_products = df_products.dropna()
df_products = df_products.set_index("12NC")
'''
df_import_supplier_A.head()

,0,7,9
0,NaN,Temperatures,Mass
1,NaN,USCS,USCS
2,NaN,°F,lb
3,NaN,32,0
4,NaN,200,2000


In [58]:
for col in df_import_supplier_A.columns.tolist():
    # Define the types as used in columns derived from database
    try:
        unit_subject = df_import_supplier_A[col].iloc[0].lower()
        unit_system = df_import_supplier_A[col].iloc[1].upper()
        unit_specs = df_import_supplier_A[col].iloc[2]
    except:
        unit_subject = "12NC"

    # Add columns and convert in place
    new_col_name = str('Supplier A '+ unit_subject)
    df_products[new_col_name] = df_import_supplier_A[col].iloc[6:df_import_supplier_A.shape[0]]

    if col > 0:
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : cleanup_data_values_return_float(x))
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : convert_units_from_dict(dict_unit_hardcoded, unit_subject, unit_system, unit_specs, x))

    elif col == 0:
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : x.replace('.',''))

df_products = df_products.set_index("Supplier A 12NC")
df_products.head()

,Supplier A temperatures,Supplier A mass
Supplier A 12NC,,
402255334708,15.000000,89.811289
402258698328,1.111111,183.704910
402249767239,25.000000,698.985842
402235950496,55.000000,311.617958
402264653803,85.555556,306.174850


In [65]:
# import
df_import_supplier_B = pd.DataFrame()
df_import_supplier_B = pd.read_csv("DataB.csv", header=None, sep=';', encoding='utf-8', low_memory=False)

# Clean up the database
# drop column
df_import_supplier_B = df_import_supplier_B.drop(columns=range(1,7), axis=1)
df_import_supplier_B = df_import_supplier_B.drop(columns=range(8,9), axis=1)
df_import_supplier_B = df_import_supplier_B.drop(columns=range(10,14), axis=1)

df_import_supplier_B.head()

,0,7,9
0,NaN,Temperatures,Mass
1,NaN,USCS,USCS
2,NaN,F,lb
3,NaN,80,0
4,NaN,24,2000


In [64]:
df_products.append(df_import_supplier_B)
print(df_products)

Supplier A temperatures  Supplier A mass
Supplier A 12NC                                          
402255334708                   15.000000        89.811289
402258698328                    1.111111       183.704910
402249767239                   25.000000       698.985842
402235950496                   55.000000       311.617958
402264653803                   85.555556       306.174850
...                                  ...              ...
402245489916                   68.888889       141.520819
402255946517                   36.111111       738.901971
402262790749                   47.777778       286.216785
402278043281                   48.333333       801.044125
402212761490                   35.555556       360.605934

[199994 rows x 2 columns]


In [66]:

# Make the conversion list
#df_products = pd.DataFrame() # This can become a loading option later
print(df_import_supplier_B.columns.tolist())
for col in df_import_supplier_B.columns.tolist():
    # Define the types as used in columns derived from database
    try:
        unit_subject = df_import_supplier_B[col].iloc[0].lower()
        unit_system = df_import_supplier_B[col].iloc[1].upper()
        unit_specs = df_import_supplier_B[col].iloc[2]
    except:
        unit_subject = "12NC"

    # Add columns and convert in place
    new_col_name = str('Supplier B '+ unit_subject)
    df_products[new_col_name] = df_import_supplier_B[col].iloc[6:df_import_supplier_B.shape[0]]

    if col > 0:
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : cleanup_data_values_return_float(x))
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : convert_units_from_dict(dict_unit_hardcoded, unit_subject, unit_system, unit_specs, x))

df_products.head()

[0, 7, 9]


,Supplier A temperatures,Supplier A mass,Supplier B 12NC,Supplier B temperatures,Supplier B mass
Supplier A 12NC,,,,,
402255334708,15.000000,89.811289,NaN,NaN,NaN
402258698328,1.111111,183.704910,NaN,NaN,NaN
402249767239,25.000000,698.985842,NaN,NaN,NaN
402235950496,55.000000,311.617958,NaN,NaN,NaN
402264653803,85.555556,306.174850,NaN,NaN,NaN


In [41]:
df_products.iloc[10]

Supplier A temperatures        30
Supplier A mass            68.946
Supplier B 12NC               NaN
Supplier B temperatures       NaN
Supplier B mass               NaN
Name: 402236436631, dtype: object

In [44]:
df_import_supplier_B

,0,7,9
0,NaN,Temperatures,Mass
1,NaN,USCS,USCS
2,NaN,F,lb
3,NaN,80,0
4,NaN,24,2000
...,...,...,...
199995,402288016975,"89,91617234",1776
199996,402287657683,"100,9163358",1287
199997,402262726583,"82,06021872",729
199998,402220423670,"53,0804307",1850


In [45]:
data_in = '74,262727.11'
data_in = data_in.replace(',','.')
data_in = False
data_in = '-74'
data_in = cleanup_data_values_return_float(data_in)
print(type(data_in))
print(data_in)
print('-----')

if type(data_in) is bool:
    print('boolean')

if type(data_in) is int or type(data_in) is float:
    print('int or float')

if type(data_in) is str:
    print('string')
    data_in = data_in.replace(',','.')
    if data_in.count('.') > 1:
        data_in = data_in.replace('.','', data_in.count('.') - 1)

print (data_in)

<class 'float'>
-74.0
-----
int or float
-74.0


In [160]:
#df_products['1'] = [1,2]

print(df_import_supplier_A.columns.tolist()[1:])

for col in df_import_supplier_A.columns.tolist()[1:]:
    # Define the types as used in columns derived from database 
    unit_subject = df_import_supplier_A[col].iloc[0].lower()
    unit_system = df_import_supplier_A[col].iloc[1].upper()
    unit_specs = df_import_supplier_A[col].iloc[2]

    # Add columns
    new_col_name = str('Supplier A '+ unit_subject)
    df_products[new_col_name] = None

df_products = df_products.set_index("12NC")
df_products.head()


[7, 9]


,Supplier A temperatures,Supplier A mass
12NC,,
12NC,None,None
402.255.334.708,None,None
402.258.698.328,None,None
402.249.767.239,None,None
402.235.950.496,None,None


In [ ]:
'''
Reserved space to use like notepad, handy to retrieve useful code
'''

## Here more code needs to come
'''
#######
#   PintPy
# convert_pintpy(1,"meter","cm") # does not work
ureg.meter
#   PyPi: Unit-converter
# #convert_unit_converter(100, kilometres, miles) # does not work
#   PyPi: Workdays
#   PyPi: BusinessHours
#######
'''

from datetime import datetime
start_time = datetime.now() # for time loop
# START TIME LOOP #
# END OF TIME LOOP #
time_elapsed = datetime.now() - start_time
print('Time elapsed (h:m:s.ms) {}'.format(time_elapsed))

In [41]:
## Trying out stuff

y = df_import_supplier_A['Temperatures']
y.iloc[2:y.size]

2          0
3         80
4          C
5         68
6         58
          ..
199994     5
199995    72
199996     5
199997    26
199998     5
Name: Temperatures, Length: 199997, dtype: object

In [ ]:
>>> from unit_converter.converter import convert, converts
>>>
>>> convert('2.78 daN*mm^2', 'mN*µm^2')
>>> Decimal('2.78E+10')
>>>
>>> converts('2.78 daN*mm^2', 'mN*µm^2')
>>> '2.78E+10'
>>>
>>> converts('78 min', 'h')
>>> '1.3'
>>>
>>> converts('52°C', '°F')
>>> '125.6'
>>>
>>> converts('120 km*h^-1', 'mile*h^-1')
>>> '74.56454306848007635409210214'

In [31]:
# General importing
import pandas as pd
from unit_converter.converter import convert, converts # copnvert returns decimal, converts returns string

dict_unit_for_pypi_unit_converter = {
    '°C':'°C',
    '°F':'°C',
    'lb':'kg'
}

# import
df_import_supplier_A = pd.DataFrame()
df_import_supplier_A = pd.read_csv("Data.csv", header=None, sep=';', encoding='utf-8', low_memory=False)

# Clean up the database
# drop column
df_import_supplier_A = df_import_supplier_A.drop(columns=range(1,7), axis=1)
df_import_supplier_A = df_import_supplier_A.drop(columns=range(8,9), axis=1)
df_import_supplier_A = df_import_supplier_A.drop(columns=range(10,14), axis=1)

df_products = pd.DataFrame()

for col in df_import_supplier_A.columns.tolist():
    # Define the types as used in columns derived from database
    try:
        unit_subject = df_import_supplier_A[col].iloc[0].lower()
        unit_system = df_import_supplier_A[col].iloc[1].upper()
        unit_specs = df_import_supplier_A[col].iloc[2]
    except:
        unit_subject = "12NC"
    
    # Add columns and convert in place
    new_col_name = str('Supplier A '+ unit_subject)
    df_products[new_col_name] = df_import_supplier_A[col].iloc[6:df_import_supplier_A.shape[0]]
    try: 
        if col > 0 :
            df_products[new_col_name] = df_products[new_col_name].apply(lambda x : convert(str(x + '' + unit_specs), dict_unit_for_pypi_unit_converter[unit_specs]))
    except:
        df_products[new_col_name] = None
        pass

df_products = df_products.set_index("Supplier A 12NC")
df_products.head()

,Supplier A temperatures,Supplier A mass
Supplier A 12NC,,
402.255.334.708,15.0000000000000000000000000,None
402.258.698.328,1.1111111111111111111111111,None
402.249.767.239,25.0000000000000000000000000,None
402.235.950.496,55.0000000000000000000000000,None
402.264.653.803,85.5555555555555555555555555,None


In [52]:
# Convert Units
########
# PyPi: Unit-Converter https://pypi.org/project/unit-convert/


dict_unit_for_pypi_unit_convert = {
    'fahrenheit':'celsius',
    'F':'celsius',
    'lb':'kg'
}

def convert_unit_converter(unit_in, x):
    from unit_convert import UnitConvert
    unit_out = dict_unit_for_pypi_unit_convert[unit_in]
    
    print(unit_in)
    print(unit_out)

    unit_in = 'fahrenheit'
    attrs = {unit_in: x}
    return getattr(UnitConvert(**attrs), unit_out)

'''
# Examplary code how to use pypi:
# Yards + kilometres to miles
x = UnitConvert(yards=136.23, kilometres=60).miles
print(x)
# Bytes to terabytes
x = UnitConvert(b=19849347813875).tb
print(x)
'''

# General importing
import pandas as pd

# import
df_import_supplier_A = pd.DataFrame()
df_import_supplier_A = pd.read_csv("Data.csv", header=None, sep=';', low_memory=False)

# Clean up the database
# drop column
df_import_supplier_A = df_import_supplier_A.drop(columns=range(1,7), axis=1)
df_import_supplier_A = df_import_supplier_A.drop(columns=range(8,9), axis=1)
df_import_supplier_A = df_import_supplier_A.drop(columns=range(10,14), axis=1)

for col in df_import_supplier_A.columns.tolist():
    # Define the types as used in columns derived from database
    try:
        unit_subject = df_import_supplier_A[col].iloc[0].lower()
        unit_system = df_import_supplier_A[col].iloc[1].upper()
        unit_specs = df_import_supplier_A[col].iloc[2]
    except:
        unit_subject = "12NC"
    
    # Add columns and convert in place
    new_col_name = str('Supplier A '+ unit_subject)
    df_products[new_col_name] = df_import_supplier_A[col].iloc[6:df_import_supplier_A.shape[0]]
    if col > 0 :
        df_products[new_col_name] = df_products[new_col_name].apply(lambda x : convert_unit_converter(unit_specs, x))

df_products = df_products.set_index("Supplier A 12NC")
df_products.head()

#x = UnitConvert(yards=136.23, kilometres=60).miles
#print(x)

def convert_unit_converter_old(data_in, unit_in, unit_out):
    from unit_convert import UnitConvert
    return(UnitConvert(unit_in=data_in).unit_out)

KeyError: '°F'

In [51]:
# Convert Units
########
# PintPy: https://pint.readthedocs.io/en/0.11/

## Error: currently this does not want to work

import pint
ureg = pint.UnitRegistry() 

'''
# Examplary code how to use pintpy: 
print(3 * ureg.meter + 4 * ureg.cm)
'''

#print(0 * ureg.meter + 4 * ureg.ft)
print(0 * ureg.meter + 2 * ureg.ft)

pint_dict = {
    'meter' : ureg.meter,
    'feet' : ureg.ft
}

def convert_pintpy(data_in, unit_in, unit_out):
    ureg = pint.UnitRegistry()

    unit_in = "ureg." + unit_in # this string conversion seems to create error
    unit_out = "ureg." + unit_out # but how to create this variable otherwise?
    out = (0 * unit_out + data_in * unit_in)

    return(out)

0.6095999999999999 meter


In [14]:
# Convert Time
########
#   PyPi: Workdays

## Needs to be made

In [15]:
# Convert Time
########
#   PyPi: BusinessHours

## Needs to be made